In [23]:
# Variable definitions
import pandas as pd
import os
lcaDataPath = "C:/Users\lukec\OneDrive - University of Cambridge\Projects\PhD\Data\LCA\EcoSpold01"

In [36]:
a = pd.read_xml(lcaDataPath+'/'+os.listdir(lcaDataPath)[0])

,0
number,3808079
generator,openLCA
timestamp,2021-12-08T16:40:03.881Z
{http://www.EcoInvent.org/EcoSpold01}metaInformation,NaN
{http://www.EcoInvent.org/EcoSpold01}flowData,NaN
